# **Cohort SV Enrichment Project**

**Author:** Shuk

**Background**
Following up on `Augusta_Layman` SV enrichment project, an analysis workflow suggestion was to calculate percent presence of a controlb SV within itself. This function will be tested and compared against an .smap file's `Present_in_%_of_BNG_control_samples`and `Present_in_%_of_BNG_control_samples_with_the_same_enzyme` columns.

**Objective**
- Implement a function that takes an SV as input, finds alignments/non-alignments and calculates percent presence in % of BNG control samples.

**Concept**
- given an SV, find similar (poswindow, reciprocalsize) SVs like it in controldb. Look into the unique Sample IDs of these similar SVs, then divide that by total number ofd unique sample IDs in controldb

**TO-DO**
1)  Use import os to improve internal file calling, no need to input root path.
2)  Case SV selection criteria: shared by >2 SVs and overlaps morbid gene. Match case to control SVs.
3)  Implement functions to gradually add new SVs into existing groups/create new SVs as new .smaps are added
4)  Add DGV bed file filter
5)  Add inversion flattening feature
6)  Make case and control SV filter summary for QC

# RUN THIS FIRST: Load Python packages, BED files, and helper functions

In [1]:
# load packages
from itertools import groupby
from matplotlib_venn import venn2, venn2_circles
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

from scipy import stats
from scipy.stats import chi2
from scipy.stats import chisquare
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import f_oneway
# from scipy.stats.contingency import odds_ratio

import copy
import re 
import os
import pathlib
working_dir = pathlib.Path().absolute()
os.chdir(working_dir)


%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Identify columns needed for output list
out_headers = ["OverlapGenes", "RefcontigID1" , "RefcontigID2", 
                "RefStartPos", "RefEndPos", "Type", "Zygosity", 
                "Present_in_%_of_BNG_control_samples", "Present_in_%_of_BNG_control_samples_with_the_same_enzyme", 
                "SVsize", "SVfreq", "Self_molecule_count"]        


In [3]:
# Helper function to get sample ID
def get_sample_name(sample_list, filename):
    # Sample ID 
    sample_ID = filename.replace("_smap_table.txt","")
    
    # check is sample ID is in sample list, return if true
    if sample_list["Sample Name"].str.contains(sample_ID).any():
        return sample_ID

# TEST
# print(get_sample_name(ntd_110_list, '17420_smap_table.txt'))

In [4]:
# Helper function to filter for SVs equal or below a certain threshold (inadvertently excludes inversion partials)
def filter_rare_svs(df, percent_cutoff):
    df = df.astype({"Present_in_%_of_BNG_control_samples":'float64', "Present_in_%_of_BNG_control_samples_with_the_same_enzyme":'float64'})

    return df.loc[((df["Present_in_%_of_BNG_control_samples"] <= percent_cutoff) & (df["Present_in_%_of_BNG_control_samples_with_the_same_enzyme"] <= percent_cutoff))
            & ((df["Present_in_%_of_BNG_control_samples"] != -1) & (df["Present_in_%_of_BNG_control_samples_with_the_same_enzyme"] != -1))]

In [5]:
def exclude_types_svs(df):
    excluded_types = ["deletion_nbase", "insertion_nbase", "gain_masked", "loss_masked", "inversion_partial", "trans_interchr_common", "trans_intrachr_common", 'translocation_interchr', 'trans_intrachr_segdupe', 'trans_interchr_segdupe', 'translocation_intrachr']
    return df.loc[~df["Type"].isin(excluded_types)]

In [6]:
# Output file for QC
# File has SVs without trans removed, n_gaps, and overlapping genes labeled
def out_file(df, name):
    df.to_csv(rf"Output\percent_presence_28Feb2023\{name}.csv", index=False)

def in_file(name):
    df = pd.read_csv(rf"Output\percent_presence_28Feb2023\{name}.csv", dtype=object, index_col=False)
    return df

# out_file(ctrldb_df, "ctrldb_df_2")

In [7]:
# ['deletion', 'insertion', 'insertion_nbase',
#        'trans_interchr_common', 'trans_intrachr_common',
#        'translocation_interchr', 'duplication', 'inversion',
#        'trans_intrachr_segdupe', 'duplication_inverted', 'deletion_nbase',
#        'inversion_paired', 'trans_interchr_segdupe', 'duplication_split',
#        'translocation_intrachr']

# Helper function to filter by confidence intervals

# 'trans', 'duplication_inverted', 'duplication','duplication_split', 'inv', 'ins', 'del'
def filter_sig_svs(df):
    dict_conf = {
        "ins": 0,
        "del": 0,
        "dup": -1,
        "inv": float(0.7),
        "trans": float(0.05) # intrachr and interchr
    }

    def get_sv_conf_thres(sv):
        sv = str(sv)
        if "ins" in sv:
            return dict_conf["ins"]
        elif "del" in sv:
            return dict_conf["del"]
        elif "duplication" in sv:
            return dict_conf["dup"]
        elif "inv" in sv:
            return dict_conf["inv"]
        elif "trans" in sv:
            return dict_conf["trans"]
        else:
            print(f"unknown sv type: {sv}")

    df = df.astype({'Confidence':'float64'})
    pass_conf = df[["Confidence", "Type"]].apply(lambda x: True if x[0] >= float(get_sv_conf_thres(x[1])) else False, axis=1)

    return df.loc[pass_conf == True]

In [8]:
# helper function to flatten inversion pairs into 1 inversion
def flatten_df_inv(df):
    # count inversion pairs and singles
    n_inv, n_pair, n_sing = 0,0,0

    # subset inversions and inversion_partials
    inv = ['inversion', 'inversion_partial']
    inv_df = copy.deepcopy(df.loc[df['Type'].isin(inv)])

    n_inv = len(inv_df)
    print("inversions & inversion_partials:", n_inv)

    # initialize empty dataframe
    out_df = pd.DataFrame()

    # proceed if there are inversions, return original df if there are none
    if len(inv_df) == 0:
        return df
    
    # remove inversions from main df
    df = df.loc[~df['Type'].isin(inv)]

    # initialize column of T/F, where SVs == T have been analyzed for flattening, and F haven't
    inv_df.loc[:, 'flattened'] =  False

    # loops through inversions and flattens inversions where LinkID matches SmapID
    while False in pd.unique(inv_df['flattened']):
        
        df_copy = copy.deepcopy(inv_df.loc[inv_df['flattened'] == False])
        # print(len(df_copy))

        # get first row item
        qry_ID = df_copy['Sample_ID'].iat[0]
        qry_chr = df_copy['chr'].iat[0]
        qry_smapID = df_copy['SmapID'].iat[0]
        qry_linkID = df_copy['LinkID'].iat[0]

        # find pair for the row if exists
        df_copy = df_copy.loc[(df_copy['Sample_ID'] == qry_ID)
                            & (df_copy['chr'] == qry_chr)
                            & ((df_copy['SmapID'] == qry_linkID)
                            | (df_copy['SmapID'] == qry_smapID))]

        # update original inv_df column as flattened
        inv_df.loc[(inv_df['Sample_ID'] == qry_ID) 
                    & (inv_df['chr'] == qry_chr) 
                    & ((inv_df['SmapID'] == qry_linkID) 
                    | (inv_df['SmapID'] == qry_smapID)), ['flattened']] = True

        # flatten if there is a pair, report only row if not
        if len(df_copy) > 1:
            
            # increment counter
            n_pair += 1

            # choose inversion in the pair, combine both
            if "inversion" in pd.unique(df_copy['Type']):
                row = df_copy.loc[(df_copy['RefEndPos'] != -1.0) & (df_copy['Type'] == 'inversion')].reset_index() # choose inversions
            else:
                row = df_copy.iloc[[0]].reset_index()
            
            # merge pairs, choosing the lowest and largest
            ref = np.concatenate((df_copy['RefStartPos'].to_numpy(), df_copy['RefEndPos'].to_numpy()), axis=None)
            ref = np.delete(ref, np.where(ref == -1))
            # print(ref)

            row.at[0, 'RefStartPos'] = np.amin(ref, axis=None)
            row.at[0, 'RefEndPos'] = np.amax(ref, axis=None)

        else:
            # increment counter
            n_sing += 1
            
            row = df_copy.iloc[[0]].reset_index()
            

        # append to out_df        
        # print(row)
        out_df = pd.concat([out_df, row], axis=0, ignore_index=True)

    # print flattened inversions for QC
    out_file(out_df, 'flattened_inv_out')

    # append out_df to df

    # return concatenated flattened inversions with original input df
    return pd.concat([out_df[df.columns], df], axis=0, ignore_index=True), n_inv, n_pair, n_sing

In [9]:
# Standardized headers for both bed files
# headers: chrom	chromStart	chromEnd	Gene	Index	strand	chromStart2	chromEnd	RGB
bed_headers = ["chr","RefStartPos","RefEndPos","Gene","Index","Strand","RefStartPos2","RefEndPos2","RGB"]

# Load hg38.knownCanonical.bed
bed_df = pd.read_csv(r"Input\ref_genes\hg38.knownCanonical.mapped_KH.bed", dtype=object, index_col=False, sep="\t")
bed_df = bed_df.astype({'RefStartPos':'float64', 'RefEndPos':'float64', 'chr':'int'})

# Load hg38_gaps.bed
gaps_df = pd.read_csv(r"Input\ref_genes\hg38_gaps.bed", dtype=object,  index_col=False, sep="\t", names=bed_headers)
gaps_df = gaps_df.astype({'RefStartPos':'float64', 'RefEndPos':'float64', 'chr':'int'})

# Load hg38 pseudogene list.bed
pseudo_df = pd.read_csv(r"Input\ref_genes\hg38_EncodeGencodePseudoGeneV41.bed",dtype=object,  index_col=False, sep="\t")
pseudo_df = pseudo_df.astype({'RefStartPos':'float64', 'RefEndPos':'float64', 'chr':'int'})

# set redstartstop as float for downstream filtering functions
for df in [bed_df, gaps_df, pseudo_df]:
    df = df.astype({'RefStartPos':'float64', 'RefEndPos':'float64', 'chr':'int'})

# display(bed_df.head(3))
# display(gaps_df.head(3))
# bed_df.shape, gaps_df.shape


In [10]:
# Helper function to remove SVs overlapping gaps bed file
def filter_genes(bed_df, chr, start, end):
    
    match_chr = bed_df.loc[(bed_df["chr"] == chr)]

    # display(match_chr)

    # transform matched chr rows into numpy (faster processing than directly panda slicing)
    bed_genes = np.array(match_chr["Gene"])
    bed_start = np.array(match_chr["RefStartPos"])
    bed_end = np.array(match_chr["RefEndPos"])

    overlap_idx = np.where((((start <= bed_start) & (bed_start <= end)) 
                            | ((bed_start <= start) & (start <= bed_end))))
    
    overlap_genes = bed_genes[overlap_idx]
    
    if len(overlap_genes) > 0:
        return ";".join(list(np.unique(overlap_genes)))
    else:
        return "-"

# filter by chr, RefStartPos, and RefEndPos
# filter_nogaps(9, 41132473.0, 41293385.0) # Test

# Helper function to remove SVs overlapping gaps bed file
def filter_nogaps(type, chr, start, end):

    # n_gaps check only on insertion and deletions
    if (("ins" in type) | ("del" in type)): 
        match_chr = gaps_df.loc[(gaps_df["chr"] == chr)]
        # check each SV related to gaps[chr"] == chr for overlaps, False if none

        # transform matched chr rows into numpy (faster processing than directly panda slicing)
        # bed_genes = np.array(match_chr["Gene"])
        bed_start = np.array(match_chr["RefStartPos"])
        bed_end = np.array(match_chr["RefEndPos"])

        for bedStart, bedEnd in zip(bed_start, bed_end):
            if (((start <= bedStart) & (bedStart <= end)) 
            | ((bedStart <= start) & (start <= bedEnd))):
                return True

    return False

# filter by chr, RefStartPos, and RefEndPos
# filter_nogaps("ins", 9, 41132473.0, 41293385.0) # Test

In [11]:
def parse_sorted_unique_genes(gene_col):
    return gene_col.str.split(';').map(lambda x: ','.join(list(set(sorted(x)))))

# Load de novo Controlsdb (Control)

This section is taken from `layman_enrichment_analysis_singlebkpt_nonmasked_500kb`

Load ctrldb for de novo assembly

In [12]:
ctrl_df = in_file('ctrl_nogaps_out').astype({'RefStartPos':'float64', 'RefEndPos':'float64', 'chr':'int', 'Size':'float64'})
ctrl_df = ctrl_df.loc[ctrl_df['Sample'].str.contains("dle1")]
ctrl_df.columns

Index(['Sample', 'Type', 'algorithm', 'chr', 'bkpt', 'chrB', 'bkptB',
       'RefStartPos', 'RefEndPos', 'Size', 'Zygosity', 'Confidence', 'smapID',
       'linkSmapId', 'has_gaps'],
      dtype='object')

In [13]:
len(ctrl_df)

2749893

In [15]:
ctrl_df['Type'].unique()

array(['duplication_inverted', 'duplication', 'duplication_split', 'inv',
       'ins', 'del'], dtype=object)

In [56]:
len(ctrl_df[ctrl_df['Sample'].str.contains('dle')]['Sample'].unique())

179

In [13]:
# ctrl_df = in_file('control_sig_out').astype({'RefStartPos':'float64', 'RefEndPos':'float64', 'chr':'int', 'Size':'float64'})
# ctrl_df.columns

In [83]:
# # Load de novo controlsdb and standardize headers
# ctrldb_df = pd.read_csv(r"Input\controlsdb\ctrl_sv_db_anonymize_hg38.txt", index_col=False, sep="\t")
# ctrldb_headers = ["Sample", "Type", "algorithm", "chr", "bkpt", "chrB", "bkptB", "RefStartPos", "RefEndPos", "Size", "Zygosity", "Confidence", "smapID", "linkSmapId"]
# ctrldb_df.columns = ctrldb_headers
# ctrldb_df = ctrldb_df.astype({'RefStartPos':'float64', 'RefEndPos':'float64', 'chr':'int', 'Size':'float64'})
# ctrldb_df = ctrldb_df.loc[~ctrldb_df['Type'].str.contains("trans")]
# ctrldb_df["has_gaps"] = ctrldb_df.apply(lambda x: filter_nogaps(x["Type"], x["chr"], x["RefStartPos"], x["RefEndPos"]), axis=1)
# ctrldb_df = ctrldb_df.loc[ctrldb_df["has_gaps"] == False]
# ctrldb_df = ctrldb_df.loc[ctrl_df['Sample'].str.contains("dle1")]


In [84]:
out_file(ctrldb_df, "ctrl_nogaps_out")

In [78]:
out_file(pd.DataFrame(ctrldb_df['Sample'].unique()), "all_ctrldb_samples")

filter for dle1 samples only

In [79]:
ctrldb_df = ctrldb_df.loc[ctrldb_df['Sample'].str.contains("dle1")]

# # filter controlsdb by score filter
# sig_df = filter_sig_svs(dle1_df)
# print(f"{len(sig_df)} SVs are significant, with confidence scores above Access recommended threshold")

# out_file(sig_df, "control_sig_out")

In [129]:
# ctrldb_df = in_file('control_sig_out').astype({'RefStartPos':'float64', 'RefEndPos':'float64', 'chr':'int', 'Size':'float64'})
# ctrldb_df.columns

Index(['Sample', 'Type', 'algorithm', 'chr', 'bkpt', 'chrB', 'bkptB',
       'RefStartPos', 'RefEndPos', 'Size', 'Zygosity', 'Confidence', 'smapID',
       'linkSmapId'],
      dtype='object')

In [145]:
ctrldb_df.columns

Index(['Sample', 'Type', 'algorithm', 'chr', 'bkpt', 'chrB', 'bkptB',
       'RefStartPos', 'RefEndPos', 'Size', 'Zygosity', 'Confidence', 'smapID',
       'linkSmapId'],
      dtype='object')

# Get % Presence in Controldb

Translocation coordinates are marked based on `bkpt` and `bkptB`

In [146]:
ctrldb_df[~ctrldb_df['bkpt'].isna()]['Type'].unique()

array(['trans'], dtype=object)

In [147]:
ctrldb_df['Type'].unique()

array(['trans', 'duplication_inverted', 'duplication',
       'duplication_split', 'inv', 'ins', 'del'], dtype=object)

In [148]:
ctrldb_df[ctrldb_df['Type'].str.contains('trans')]['Type'].unique()

array(['trans'], dtype=object)

In [149]:
ctrldb_df.head()

,Sample,Type,algorithm,chr,bkpt,chrB,bkptB,RefStartPos,RefEndPos,Size,Zygosity,Confidence,smapID,linkSmapId
0,1_dle1_sop,trans,assembly_comparison,8,7359682.0,8.0,7950679.0,NaN,NaN,NaN,heterozygous,0.01,7408,NaN
1,1_dle1_sop,trans,assembly_comparison,8,7359682.0,8.0,7950679.0,NaN,NaN,NaN,heterozygous,0.01,7409,NaN
2,1_dle1_sop,trans,assembly_comparison,8,7913502.0,8.0,7398088.0,NaN,NaN,NaN,heterozygous,0.00,7417,NaN
3,1_dle1_sop,trans,assembly_comparison,8,7807846.0,8.0,7513171.0,NaN,NaN,NaN,homozygous,0.01,7421,NaN
4,1_dle1_sop,trans,assembly_comparison,8,7913502.0,8.0,7398088.0,NaN,NaN,NaN,heterozygous,0.00,7422,NaN


In [77]:
# Helper function to calculate % presence in controldb
ref_ctrl_df = in_file('ctrl_nogaps_out').astype({'RefStartPos':'float64', 'RefEndPos':'float64', 'chr':'int', 'Size':'float64'})
# ref_ctrl_df = in_file('control_sig_out').astype({'RefStartPos':'float64', 'RefEndPos':'float64', 'chr':'int', 'Size':'float64'})
# ref_ctrl_df =pd.read_csv(r"Input\controlsdb\ctrl_sv_db_anonymize_hg38.txt", index_col=False, sep="\t")
# ctrldb_headers = ["Sample", "Type", "algorithm", "chr", "bkpt", "chrB", "bkptB", "RefStartPos", "RefEndPos", "Size", "Zygosity", "Confidence", "smapID", "linkSmapId"]
# ref_ctrl_df.columns = ctrldb_headers
# ctrldb_df = ctrldb_df.astype({'RefStartPos':'float64', 'RefEndPos':'float64', 'chr':'int','Size':'float64'})

ref_ctrl_df = ref_ctrl_df.loc[ref_ctrl_df['Sample'].str.contains("dle1")]
ref_ctrl_df = ref_ctrl_df.loc[~ref_ctrl_df['Type'].str.contains("trans")]
ref_ctrl_df.columns

ctrl_db_n = len(ref_ctrl_df.loc[ref_ctrl_df['Sample'].str.contains("dle1")]['Sample'].unique())
print(ctrl_db_n)

def calculate_percent_presence_controldb(Type, zygo, chrA, chrB, start, end, size):

    def get_sv_type(sv):
        sv = str(sv)
        if "ins" in sv:
            return "ins"
        elif "del" in sv:
            return "del"
        elif "duplication" in sv:
            return sv
        elif "inv" in sv:
            return "inv"
        elif "trans" in sv:
            return "trans"
        else:
            print(f"unknown sv type: {sv}")

    # transform matched chr rows into numpy (faster processing than directly panda slicing)
    # bed_genes = np.array(match_chr["Gene"])

    Type = get_sv_type(Type)

    ins_del_position_overlap=10000
    ins_del_size_percent_similarity=50
    inversion_position_overlap=50000
    translocation_position_overlap=50000
    duplication_position_overlap=10000
    duplication_size_percent_similarity=50

    if (Type == 'ins') | (Type == 'del'):
        posWindow, reciprocalSize = ins_del_position_overlap, ins_del_size_percent_similarity
    elif 'duplication' in Type:
        posWindow, reciprocalSize = duplication_position_overlap, duplication_size_percent_similarity
    elif Type == 'inv':
        posWindow, reciprocalSize = inversion_position_overlap, 0
    elif Type == 'trans':  
        posWindow, reciprocalSize = translocation_position_overlap, 0

    bed_df = ref_ctrl_df.copy()

    # if interchr_translocation
    if Type == 'trans':
        print(Type)
        if chrA == chrB:
        
            match_chr = bed_df.loc[(bed_df["chr"] == chrA) & (bed_df['Type'] == Type)]

            bed_sample = np.array(match_chr['Sample'])
            bed_start = np.array(match_chr["bkpt"])
            bed_end = np.array(match_chr["bkptB"])

            overlap_idx = np.where((((start - posWindow <= bed_start) & (bed_start <= end + posWindow)) \
                                    | ((bed_start <= start - posWindow) & (start - posWindow <= bed_end))))

            percent_presence = 100 * len(list(match_chr.iloc[overlap_idx]['Sample'].unique()))/ctrl_db_n
            
        else:
            match_chrA = bed_df.loc[(bed_df["chr"] == chrA) & (bed_df['Type'] == Type)]
            match_chrB = bed_df.loc[(bed_df["chr"] == chrB) & (bed_df['Type'] == Type)]

            bed_start = np.array(match_chrA["bkpt"])
            bed_end = np.array(match_chrB["bkptB"])

            overlap_idxA = np.where((((start - posWindow <= bed_start) & (bed_start <= end + posWindow)) \
                                    | ((bed_start <= start - posWindow) & (start - posWindow <= bed_end))))
            
            overlap_idxB = np.where((((start - posWindow <= bed_start) & (bed_start <= end + posWindow)) \
                                    | ((bed_start <= start - posWindow) & (start - posWindow <= bed_end))))
            
            samples = pd.concat([match_chrA.iloc[overlap_idxA]['Sample'], match_chrB.iloc[overlap_idxB]['Sample']], axis=0, ignore_index=True)
            n = list(samples.unique())
            display(n)
            percent_presence = 100 * len(n)/ctrl_db_n
            
    elif Type == 'inv':
        print(Type)
        
        match_chr = bed_df.loc[(bed_df["chr"] == chrA) & (bed_df['Type'] == Type)]

        bed_sample = np.array(match_chr['Sample'])
        bed_start = np.array(match_chr["RefStartPos"])
        bed_end = np.array(match_chr["RefEndPos"])
        bed_size = np.array(match_chr["Size"])

        overlap_idx = np.where((((start - posWindow <= bed_start) & (bed_start <= end + posWindow)) \
                                | ((bed_start <= start - posWindow) & (start - posWindow <= bed_end))))
        
        percent_presence = 100 * len(np.unique(bed_sample[overlap_idx]))/ctrl_db_n
    # if indeldup
    else:
        print(Type)
        
        match_chr = bed_df.loc[(bed_df["chr"] == chrA) & (bed_df['Type'] == Type)]

        bed_sample = np.array(match_chr['Sample'])
        bed_start = np.array(match_chr["RefStartPos"])
        bed_end = np.array(match_chr["RefEndPos"])
        bed_size = np.array(match_chr["Size"])

        overlap_idx = np.where((((start - posWindow <= bed_start) & (bed_start <= end + posWindow)) \
                                | ((bed_start <= start - posWindow) & (start - posWindow <= bed_end)))
                                & (((size/bed_size)*100 >= reciprocalSize) \
                                    & ((bed_size/size)*100 >= reciprocalSize)))
        
        # display(match_chr.iloc[overlap_idx])
        percent_presence = 100 * len(np.unique(bed_sample[overlap_idx]))/ctrl_db_n

    # display(list(match_chr.iloc[overlap_idx]['Sample'].unique()))
    return percent_presence

179


In [75]:
ref_ctrl_df.head()

,Sample,Type,algorithm,chr,bkpt,chrB,bkptB,RefStartPos,RefEndPos,Size,Zygosity,Confidence,smapID,linkSmapId
0,1_dle1_sop,trans,assembly_comparison,2,134647984.0,2.0,140754758.0,NaN,NaN,NaN,heterozygous,0.05,2126,NaN
1,1_dle1_sop,trans,assembly_comparison,1,12938769.0,1.0,13263012.0,NaN,NaN,NaN,heterozygous,0.32,105,NaN
2,1_dle1_sop,trans,assembly_comparison,1,12938769.0,1.0,13263012.0,NaN,NaN,NaN,heterozygous,0.3,158,NaN
3,1_dle1_sop,trans,assembly_comparison,1,12938769.0,1.0,13263012.0,NaN,NaN,NaN,heterozygous,0.07,184,NaN
4,1_dle1_sop,trans,assembly_comparison,1,13005792.0,1.0,13223159.0,NaN,NaN,NaN,heterozygous,0.33,188,NaN


Load case cleaned file for testing

In [59]:
case_df = in_file('case_compiled_out')
case_df.rename(columns={"RefcontigID2":"chrB"}, inplace=True)
case_df.rename(columns={"RefcontigID1":"chr"}, inplace=True)
case_df = case_df.astype({'RefStartPos':'float64', 'RefEndPos':'float64', 'chr':'int', 'chrB':'int', 'num_overlap_DGV_calls':'int', 'SVsize':'float64', 'Present_in_%_of_BNG_control_samples_with_the_same_enzyme':'float64'})

In [53]:
case_df[case_df['SVsize'] == 0]['Type'].unique()

array([], dtype=object)

In [ ]:
for type in case_df['Type'].unique():
    if 'trans' in type:
        continue
    # df = case_df.loc[(case_df['Type'] == type) & (case_df['Present_in_%_of_BNG_control_samples_with_the_same_enzyme'] <= 1.0)].head()
    df = case_df.loc[(case_df['Type'] == type)].head()
    df['%_controldb'] = df[['Type', 'Zygosity','chr', 'chrB', 'RefStartPos', 'RefEndPos', 'SVsize']].apply(lambda x: calculate_percent_presence_controldb(x[0], x[1], x[2], x[3], x[4], x[5], x[6]), axis=1)
    print(type), display(df)

In [ ]:
rare_df = case_df.loc[(case_df['Present_in_%_of_BNG_control_samples_with_the_same_enzyme'] <= 1.0)].copy()
rare_df['%_controldb'] = rare_df[['Type', 'Zygosity','chr', 'chrB', 'RefStartPos', 'RefEndPos', 'SVsize']].apply(lambda x: calculate_percent_presence_controldb(x[0], x[1], x[2], x[3], x[4], x[5], x[6]), axis=1)
out_file(case_df, 'test_percent_controldb_out')

# Filtering Controldb

In [14]:
# Load de novo controlsdb and standardize headers
ctrldb_df = pd.read_csv(r"Input\controlsdb\ctrl_sv_db_anonymize_hg38.txt", index_col=False, sep="\t")
ctrldb_headers = ["Sample", "Type", "algorithm", "chr", "bkpt", "chrB", "bkptB", "RefStartPos", "RefEndPos", "Size", "Zygosity", "Confidence", "smapID", "linkSmapId"]
ctrldb_df.columns = ctrldb_headers
ctrldb_df = ctrldb_df.astype({'RefStartPos':'float64', 'RefEndPos':'float64', 'chr':'int','Size':'float64'})
# ctrldb_df = ctrldb_df.loc[~ctrldb_df['Type'].str.contains("trans")]
# ctrldb_df["has_gaps"] = ctrldb_df.apply(lambda x: filter_nogaps(x["Type"], x["chr"], x["RefStartPos"], x["RefEndPos"]), axis=1)
# ctrldb_df = ctrldb_df.loc[ctrldb_df["has_gaps"] == False]
# ctrldb_df = ctrldb_df.loc[ctrl_df['Sample'].str.contains("dle1")]


In [20]:
ctrldb_df['Type'].unique()

array(['trans', 'duplication_inverted', 'duplication',
       'duplication_split', 'inv', 'ins', 'del'], dtype=object)

In [22]:
ctrldb_df.loc[(ctrldb_df['Type'] == 'trans') & (ctrldb_df['chr'] == ctrldb_df['chrB']), 'chrB'].unique()

array([ 8.,  2.,  1., 23., 22., 15., 14., 10.,  9., 21., 20.,  7.,  5.,
       11., 17.,  6., 24., 16.,  4., 13., 12., 19.])

In [27]:
ctrldb_df.loc[(ctrldb_df['Type'].str.contains('duplication')), 'Zygosity'].unique()

array(['unknown'], dtype=object)

In [26]:
for type in ctrldb_df['Type'].unique():
    df = ctrldb_df.loc[ctrldb_df['Type'] == type].head()
    df['%_controldb'] = df[['Type', 'chr', 'chrB', 'RefStartPos', 'RefEndPos', 'Size']].apply(lambda x: calculate_percent_presence_controldb(x[0], x[1], x[2], x[3], x[4], x[5]), axis=1)
    print(type), display(df)
    

trans
trans
trans
trans
trans
trans


,Sample,Type,algorithm,chr,bkpt,chrB,bkptB,RefStartPos,RefEndPos,Size,Zygosity,Confidence,smapID,linkSmapId,%_controldb
0,1_dle1_sop,trans,assembly_comparison,8,7359682.0,8.0,7950679.0,NaN,NaN,NaN,heterozygous,0.01,7408,NaN,0.0
1,1_dle1_sop,trans,assembly_comparison,8,7359682.0,8.0,7950679.0,NaN,NaN,NaN,heterozygous,0.01,7409,NaN,0.0
2,1_dle1_sop,trans,assembly_comparison,8,7913502.0,8.0,7398088.0,NaN,NaN,NaN,heterozygous,0.00,7417,NaN,0.0
3,1_dle1_sop,trans,assembly_comparison,8,7807846.0,8.0,7513171.0,NaN,NaN,NaN,homozygous,0.01,7421,NaN,0.0
4,1_dle1_sop,trans,assembly_comparison,8,7913502.0,8.0,7398088.0,NaN,NaN,NaN,heterozygous,0.00,7422,NaN,0.0


duplication_inverted
duplication_inverted
duplication_inverted
duplication_inverted
duplication_inverted
duplication_inverted


,Sample,Type,algorithm,chr,bkpt,chrB,bkptB,RefStartPos,RefEndPos,Size,Zygosity,Confidence,smapID,linkSmapId,%_controldb
37850,1_dle1_sop,duplication_inverted,assembly_comparison,1,NaN,NaN,NaN,13223159.0,13263012.0,39854.0,unknown,-1.0,169,NaN,43.016760
37851,1_dle1_sop,duplication_inverted,assembly_comparison,1,NaN,NaN,NaN,16566162.0,16616818.0,50657.0,unknown,-1.0,171,NaN,97.765363
37852,1_dle1_sop,duplication_inverted,assembly_comparison,1,NaN,NaN,NaN,16566162.0,16616818.0,50657.0,unknown,-1.0,172,NaN,97.765363
37854,1_dle1_sop,duplication_inverted,assembly_comparison,1,NaN,NaN,NaN,16566162.0,16653300.0,87139.0,unknown,-1.0,194,NaN,97.765363
37855,1_dle1_sop,duplication_inverted,assembly_comparison,1,NaN,NaN,NaN,16566162.0,16653300.0,87139.0,unknown,-1.0,195,NaN,97.765363


duplication
duplication
duplication
duplication
duplication
duplication


...

,Sample,Type,algorithm,chr,bkpt,chrB,bkptB,RefStartPos,RefEndPos,Size,Zygosity,Confidence,smapID,linkSmapId,%_controldb
72899,1_dle1_sop,inv,assembly_comparison,1,NaN,NaN,NaN,16616818.0,16883281.0,-1.0,heterozygous,0.00,173,174.0,100.0
72900,1_dle1_sop,inv,assembly_comparison,1,NaN,NaN,NaN,16616818.0,16859181.0,-1.0,heterozygous,0.00,186,187.0,100.0
72901,1_dle1_sop,inv,assembly_comparison,1,NaN,NaN,NaN,16653300.0,16859181.0,-1.0,homozygous,0.02,192,193.0,100.0
72902,1_dle1_sop,inv,assembly_comparison,1,NaN,NaN,NaN,16653300.0,16653300.0,-1.0,heterozygous,0.00,196,197.0,100.0
72903,1_dle1_sop,inv,assembly_comparison,1,NaN,NaN,NaN,16653300.0,16859181.0,-1.0,homozygous,0.00,199,200.0,100.0


ins
ins
ins
ins
ins
ins


,Sample,Type,algorithm,chr,bkpt,chrB,bkptB,RefStartPos,RefEndPos,Size,Zygosity,Confidence,smapID,linkSmapId,%_controldb
124814,1_dle1_sop,ins,assembly_comparison,1,NaN,NaN,NaN,365535.0,387230.0,10175.0,heterozygous,0.99,3,NaN,13.407821
124816,1_dle1_sop,ins,assembly_comparison,1,NaN,NaN,NaN,663861.0,672078.0,536.0,heterozygous,0.82,5,NaN,20.670391
124817,1_dle1_sop,ins,assembly_comparison,1,NaN,NaN,NaN,710358.0,711832.0,336.0,heterozygous,-1.00,6,NaN,86.033520
124818,1_dle1_sop,ins,assembly_comparison,1,NaN,NaN,NaN,788500.0,803475.0,2436.0,heterozygous,0.99,7,NaN,100.000000
124820,1_dle1_sop,ins,assembly_comparison,1,NaN,NaN,NaN,858481.0,882186.0,2208.0,heterozygous,0.99,9,NaN,95.530726


del
del
del
del
del
del


,Sample,Type,algorithm,chr,bkpt,chrB,bkptB,RefStartPos,RefEndPos,Size,Zygosity,Confidence,smapID,linkSmapId,%_controldb
124815,1_dle1_sop,del,assembly_comparison,1,NaN,NaN,NaN,0.0,588217.0,573534.0,heterozygous,0.99,4,NaN,0.000000
124819,1_dle1_sop,del,assembly_comparison,1,NaN,NaN,NaN,817708.0,825653.0,1119.0,heterozygous,0.99,8,NaN,0.558659
124821,1_dle1_sop,del,assembly_comparison,1,NaN,NaN,NaN,896304.0,912049.0,377.0,homozygous,-1.00,10,NaN,35.754190
124822,1_dle1_sop,del,assembly_comparison,1,NaN,NaN,NaN,0.0,588217.0,573520.0,heterozygous,0.99,11,NaN,0.000000
124826,1_dle1_sop,del,assembly_comparison,1,NaN,NaN,NaN,817708.0,825653.0,1119.0,heterozygous,0.99,15,NaN,0.558659


In [97]:
ctrldb_df.head()

,Sample,Type,algorithm,chr,bkpt,chrB,bkptB,RefStartPos,RefEndPos,Size,Zygosity,Confidence,smapID,linkSmapId
0,1_dle1_sop,trans,assembly_comparison,8,7359682.0,8.0,7950679.0,NaN,NaN,NaN,heterozygous,0.01,7408,NaN
1,1_dle1_sop,trans,assembly_comparison,8,7359682.0,8.0,7950679.0,NaN,NaN,NaN,heterozygous,0.01,7409,NaN
2,1_dle1_sop,trans,assembly_comparison,8,7913502.0,8.0,7398088.0,NaN,NaN,NaN,heterozygous,0.00,7417,NaN
3,1_dle1_sop,trans,assembly_comparison,8,7807846.0,8.0,7513171.0,NaN,NaN,NaN,homozygous,0.01,7421,NaN
4,1_dle1_sop,trans,assembly_comparison,8,7913502.0,8.0,7398088.0,NaN,NaN,NaN,heterozygous,0.00,7422,NaN


In [100]:
# Initialize dictionary to keepo track of filtering
ctrldb_filtering = {}

# Main code to clean up hg38 controlsdb .smap file
print(f"starting with {len(ctrldb_df)} total SVs")

# Keep only SVs detected with 'dle1' enzyme
dle_df = ctrldb_df.loc[ctrldb_df["Sample"].str.contains("dle1")]
print(f"{len(dle_df)} SVs from dle1 labeling")

# remove translocations
notrans_df = dle_df.loc[~dle_df['Type'].str.contains("trans")]
print(f"{len(notrans_df)} SVs after translocations removed")

# # filter controlsdb by score filter
# sig_df = filter_sig_svs(notrans_df)
# print(f"{len(sig_df)} SVs are significant, with confidence scores above Access recommended threshold")

# out_file(sig_df, "control_sig_out")

# remove indel_ngaps
# True if overlaps gaps, false if not, remove Falses
notrans_df["has_gaps"] = notrans_df.apply(lambda x: filter_nogaps(x["Type"], x["chr"], x["RefStartPos"], x["RefEndPos"]), axis=1)
nogaps_df = notrans_df.loc[notrans_df["has_gaps"] == False]

out_file(nogaps_df, "control_nogaps_out")

nogaps_df['Present_in_%_of_BNG_control_samples_with_the_same_enzyme'] = nogaps_df[['Type', 'chr', 'chrB', 'RefStartPos', 'RefEndPos', 'Size']].apply(lambda x: calculate_percent_presence_controldb(x[0], x[1], x[2], x[3], x[4], x[5]), axis=1)
rare_df = nogaps_df.loc[nogaps_df['Present_in_%_of_BNG_control_samples_with_the_same_enzyme'] <= 1.0]
out_file(rare_df, "control_rare_out")

# filter controlsdb by score filter
sig_df = filter_sig_svs(rare_df)
print(f"{len(sig_df)} SVs are significant, with confidence scores above Access recommended threshold")

out_file(sig_df, "control_sig_out")

num_true = len(sig_df[sig_df["has_gaps"] == True])
num_false = len(sig_df[sig_df["has_gaps"] == False])
print(f"{num_true} has gaps, and {num_false} has no gaps, for a total of {num_true + num_false} SVs")

starting with 3428999 total SVs
2822201 SVs from dle1 labeling
2787424 SVs after translocations removed


KeyboardInterrupt: 

In [ ]:
out_file(nogaps_df, "control_nogaps_percentP{resence_out")

In [ ]:
# find overlap genes matching hg38knownCanonical.bed
nogaps_df = in_file("control_nogaps_sig_out")
nogaps_df = nogaps_df.astype({'RefStartPos':'float64','RefEndPos':'float64','chr':'int'})
nogaps_df.dtypes

nogaps_df["OverlapGenes"] = nogaps_df.apply(lambda x: filter_genes(bed_df, x["chr"], x["RefStartPos"], x["RefEndPos"]), axis=1)
out_file(nogaps_df, "control_gene_nogaps_sig_out")

no_overlaps = len(nogaps_df.loc[nogaps_df["OverlapGenes"] == "-"])
overlaps = len(nogaps_df.loc[nogaps_df["OverlapGenes"] != "-"])
print(f"From {no_overlaps + overlaps} cleaned SVs, {no_overlaps} SVs do not overlap any gene, while {overlaps} do.")

out_file(nogaps_df, "control_cleaned_out")

KeyboardInterrupt: 

In [ ]:
# # Load files and Clean Pseudogenes bed file
# # headers: chrom	chromStart	chromEnd	Gene	Index	strand	chromStart2	chromEnd	RGB
# # bed_headers = ["chr","RefStartPos","RefEndPos","Gene","Index","Strand","RefStartPos2","RefEndPos2","RGB"]

# # Load cleaned ctrl_db file
# # nogaps_df = pd.read_csv(r"Output\output_04Jan2023\control_gene_nogaps_sig_out.csv", index_col=False)
# nogaps_df = in_file('control_gene_nogaps_sig_out')
# nogaps_df = nogaps_df.astype({'RefStartPos':'float64','RefEndPos':'float64','chr':'int'})
# display(nogaps_df.head())

# # Check if SVs overlap which genes
# nogaps_df["OverlapPseudogenes"] = nogaps_df.apply(lambda x: filter_genes(pseudo_df, x["chr"], x["RefStartPos"], x["RefEndPos"]), axis=1)

# # remove SVs that overlap only pseudogenes
# overlap_pseudogenes_df = nogaps_df.loc[((nogaps_df["OverlapGenes"].str.split(';') == nogaps_df["OverlapPseudogenes"].str.split(';')) & (nogaps_df["OverlapGenes"] != "-"))
#                                     | ((nogaps_df["OverlapPseudogenes"] != "-") & (nogaps_df["OverlapGenes"] == "-"))]
# out_file(overlap_pseudogenes_df, "control_pseudogenes_out")

# # Exclude SVs that are only overlapped by pseudogenes
# cleaned_df = nogaps_df.loc[~(((nogaps_df["OverlapGenes"].str.split(';') == nogaps_df["OverlapPseudogenes"].str.split(';')) & (nogaps_df["OverlapGenes"] != "-")) 
#                             |((nogaps_df["OverlapGenes"] == "-") & (nogaps_df["OverlapPseudogenes"] != "-")))]
# # # Exclude SVs that are only overlapped by pseudogenes
# # cleaned_df = case_df.loc[~((case_df["OverlapGenes"] == case_df["OverlapPseudogenes"]) & (case_df["OverlapGenes"] != "-") 
# #                         | ((case_df["OverlapGenes"] == "-") & (case_df["OverlapPseudogenes"] != "-")))]

# print(f"There are {len(nogaps_df)} SVs before cleaning, {len(overlap_pseudogenes_df)} SVs are found to overlap only pseudogenes, leaving {len(cleaned_df)} SVs overlapping no genes or knownCanonical")

# # Excluded pseudogenes from controlsdb file
# out_file(cleaned_df, "control_cleaned_out")

In [ ]:
# Load cleaned ctrl_db file
# cleaned_df = pd.read_csv(r"Output\output_06Jan2023\control_cleaned_out.csv", index_col=False)
# cleaned_df = in_file('control_cleaned_out')
# cleaned_df.shape

In [ ]:
# Append to dictionary
ctrldb_filtering={}
ctrldb_filtering["Total SVs"] = len(ctrldb_df)
ctrldb_filtering["DLE1 Labelled SVs"] = len(dle_df)
print(pd.unique(dle_df['Type']))
ctrldb_filtering["No translocations"] = len(notrans_df)
ctrldb_filtering["Significant SVs"] = len(sig_df)
print(pd.unique(sig_df['Type']))
ctrldb_filtering["No n-base gaps"] = (len(nogaps_df))
print(pd.unique(nogaps_df['Type']))
# ctrldb_filtering["Removed Pseudogene-only Overlaps"] = (len(cleaned_df))
ctrldb_filtering

['trans' 'duplication_inverted' 'duplication' 'duplication_split' 'inv'
 'ins' 'del']
['duplication_inverted' 'duplication' 'duplication_split' 'inv' 'ins'
 'del']
['duplication_inverted' 'duplication' 'duplication_split' 'inv' 'ins'
 'del']


{'Total SVs': 3428999,
 'DLE1 Labelled SVs': 2822201,
 'No translocations': 2787424,
 'Significant SVs': 1193403,
 'No n-base gaps': 1156665}